# Implementing Recommender Systems for BoardGamesGeek.com

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
#line-by-line runtime comparison for easier code optimization.
%load_ext line_profiler

pd.set_option('display.max_rows',1000)

elite = pd.read_csv('../inputs/boardgame-elite-users.csv')
elite = elite.rename(columns = {'Compiled from boardgamegeek.com by Matt Borthwick':'UserID'})
titles = pd.read_csv('../inputs/boardgame-titles.csv')
titles = titles.rename(columns={'boardgamegeek.com game ID':'gameID'})
frequent = pd.read_csv('../inputs/boardgame-frequent-users.csv')
frequent = frequent.rename(columns = {'Compiled from boardgamegeek.com by Matt Borthwick':'UserID'})
#load up the big dataset
#users = pd.read_csv('../inputs/boardgame-users.csv')
#users = users.rename(columns = {'Compiled from boardgamegeek.com by Matt Borthwick':'UserID'})


## Baseline Predictor
This is the simplest predictor I'm making for the project. It doubles as a way to normalize the Ratings matrix R for more complex algorithms (like SVD) that require some kind of a way to fill missing ratings in the sparse matrix. Subtracting the baseline prediction from each value in R normalizes so that missing values can be set to 0.

All this prediction does is use the user rating averages, total average rating, and average item ratings to come up with a believable first guess. Details are in section 2.1 of the paper linked in the exploratory notebook. 

In [3]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split

class Base_Predictor(BaseEstimator,RegressorMixin):
    def __init__(self, DAMPENING_TERM=25, dampening=False):
        self._dampening_term = 25
        self._dampening = dampening
        
    def fit(self, X, y):
        self._mean = y.mean()
        self._R = pd.concat([X,y],axis=1).pivot_table(index='UserID',columns='gameID',values='rating')
        self._bu = self._R.apply(lambda row: self._user_base(row), axis=1)
        self._bi = self._R.apply(lambda column: self._item_base(column))
        
    def _user_base(self, row):
        """(1/M+d)*bu is with the dampening factor. Without it's 1/M * bu. To find the way to add the 
        dampening factor as a scalar multiplication: 
            k*1/M(bu) = 1/M+d(bu)
            k = M/M+d"""
        bu = row.mean() - self._mean
        if self._dampening:
            num_items_user_reviewed = row[row.notnull()].size
            damp_factor = num_items_user_reviewed/(num_items_user_reviewed+self._dampening_term)
            bu*=damp_factor
        return bu
    
    def _item_base(self, column):
        users_that_reviewed_this_item = column[column.notnull()]
        bu_for_users_that_reviewed_i = self._bu[users_that_reviewed_this_item.index].mean()
        bi = users_that_reviewed_this_item.mean()-bu_for_users_that_reviewed_i-self._mean
        if self._dampening:
            num_users_reviewed_item = column[column.notnull()].size
            damp_factor = num_users_reviewed_item/(num_users_reviewed_item+self._dampening_term)
            bi*=damp_factor
        return bi
        
    def predict(self, X):
        return self._mean + self._bu[X.UserID].values + self._bi[X.gameID].values
    

### A little Unit Testing for Sanity's sake.
I just made a simple 3x3 test data set with one missing value. I ran through the calculations by hand, and set up a little battery of tests to make sure it all works. I print out the 3 pieces of info so you can see visually. There's TDD_test_X, the user, item pair I predict. TDD_train_X is the list of values at the bottom, the middle matrix is TDD_test_X blown up into the ratings matrix (with the missing value I'm testing for showing). As long as this cell compiles without triggering an assertion error, things are working fine.

In [4]:
np.random.seed(42)

test_ratings_matrix = pd.DataFrame(np.random.randint(1,10,size=(3,3)),columns=map(int,list('456')),index=map(int,list('123')))
test_ratings_matrix.loc[3,6] = np.NaN
#collapse test frame down the same format as our dataset. 3 columns, user
TDD_train_X = test_ratings_matrix.stack().reset_index()
TDD_train_X.columns = ['UserID','gameID','rating']
TDD_test_X = pd.DataFrame(data={'UserID':[3],'gameID':[6]})
display(TDD_test_X)
display(test_ratings_matrix)
display(TDD_train_X)

def test_baseline_predictor():
    predictor = Base_Predictor()
    predictor.fit(TDD_train_X[['UserID','gameID']],TDD_train_X.rating)
    assert predictor._mean == 6.125, "The incorrect mean was calculated for the baseline test set"
    assert predictor._bu.tolist() == [0.20833333333333304,-1.125,1.375], "The wrong bu values were calculated for the baseline test"
    assert predictor._bi.tolist() == [0.05555555555555536, 0.05555555555555536, -0.16666666666666607], "incorrect bi was calculated for baseline test"
    assert predictor.predict(TDD_test_X).tolist() == [7.333333333333334], "baseline prediction for the test value was incorrect"
        

test_baseline_predictor()

,UserID,gameID
0,3,6


,4,5,6
1,7,4,8.0
2,5,7,3.0
3,7,8,NaN


,UserID,gameID,rating
0,1,4,7.0
1,1,5,4.0
2,1,6,8.0
3,2,4,5.0
4,2,5,7.0
5,2,6,3.0
6,3,4,7.0
7,3,5,8.0


## User-User Collaborative Filtering

This is the system described in section 2.2 of the linked paper. The idea is that to predict the rating of user U and item I, you use the normalized average rating of the N most similar users to U who have reviewed item I. There are multiple similarity measures that can be used, and several other hyper paramters that can be tweaked that can be fed into the class constructor for testing and comparison. 

In [5]:
from sklearn.base import BaseEstimator, RegressorMixin

def mean_normalize(row):
    return row - row.mean()

#mean normalize has already been used if this is ever applied.
def z_normalize(row):
    return row/row.std()

class U_U_predictor(BaseEstimator, RegressorMixin):
    
    #ratings matrix from the actual training values
    #_R 
    
    #precomputing user's average ratings and std to save time later.
    #_user_average_rating
    #_user_standard_deviation
    
    #user similarity matrix (size user x user)
    #_S
    
    #function that changes depending on selected similarity metric (cosine, pearson, spearman, etc.)
    #_calculate_user_similarity
    
    #switches between equation 2.6 and 2.7 in the paper
    #_normalize_to_z_scores
    
    #the paper suggests a dampening threshhold to keep users from sparse reviews getting rated as overly similar
    #_pearson_threshold
    
    #how many nearest neighbors to look at when computing rating predictions
    #_N_similar
        
    def __init__(self, similarity_type = 'pearson', normalize_to_z_scores=False, 
                 pearson_threshold=50, N_similar=10, use_negative_similarities = False):
        self.normalize_to_z_scores = normalize_to_z_scores
        self.pearson_threshold = pearson_threshold
        self.N_similar = N_similar
        self.similarity_type = similarity_type
        self.use_negative_similarities = use_negative_similarities

            
    def fit(self,X,y):
        if self.similarity_type == 'cosine':
            self._calculate_user_similarity = self._cosine_similarity
        else: self._calculate_user_similarity = self._pearson_r_similarity_vectorized
        
        self._calculate_ratings_matrix(X,y)
        self._calculate_user_similarity_matrix_s_vector()
        
        if self.normalize_to_z_scores:
            self._R = self._R.apply(z_normalize, axis=1)
            
    
    def _calculate_ratings_matrix(self, X,y):
        """Calculates the n_user x n_user similarity matrix: _S"""
        self._R = pd.concat([X,y],axis=1).pivot_table(index='UserID',columns='gameID',values='rating')
        
        #preprocessing to make user similarities easier to calculate
        self._user_average_rating = self._R.mean(axis=1)
        self._user_standard_deviation = self._R.std(axis=1)
        
        #normalize
        self._R = self._R.apply(mean_normalize, axis=1)
        
        
    def _calculate_user_similarity_matrix_s_vector(self):
        """top level user similarity _S calculation function. To save time I initially set it up as
        an N*(N_1)/2 triangle calculation instead of doing the full N^2 user, user calculation. Given
        the current vector implementation, this may be a fair bit of obfuscation for a small amount of
        benefit, it may be worth timing both implementations to see how time is actually being saved."""
        
        #initialize our similarity matrix _S, and our temp numpy matrix we'll be using while calculating.
        self._S = pd.DataFrame(index=self._R.index, columns = self._R.index.rename('User_Prime_ID'), data=0.0)
        temp = np.full(self._S.shape,np.nan)
        
        #We have a user x user matrix of similarity values, but we don't need to do the main diagonal (user1 x user1
        #will always have 1.0) and since the top and bottom diagonals are identical (since user1xuser2 = user2xuser1)
        #we only bother calculating along the upper triangle. We go row by row, the row sizes decrease as we go.
        for index,user1 in enumerate(self._R.index[:-1]):
            user2s = self._R.index[index+1:].copy()
            temp[index,index+1:] = self._calculate_user_similarity(user1,index,user2s)
            
        #now that we have the upper triangle values, all we have to do is mirror it to the bottom and we're done.
        i_lower = np.tril_indices(temp.shape[0], -1)
        temp[i_lower] = temp.T[i_lower] 
        #turn our numpy temp matrix back into a dataframe.
        self._S = pd.DataFrame(temp, columns=self._S.columns, index=self._S.index)
        #I decided I wanted 0.0 for user x user similarity, not NA.
        self._S = self._S.fillna(0.0)
        
            
    def _pearson_r_similarity_vectorized(self,user1,user1_index,user2s):
        """Calculates user1 similarity with other users
        Parameters
        ----------
        user1 : int identifier for a user
        user1_index : index position for where user1 appears in the index of _R
        user2s : array, shape [n_users]
        
        Returns
        -------
        ret : array of float, shape = [n_users]. 
            corresponds to the ordered similarity values for each user 2 in user1's 
            row of the similarity matrix.
        """
        
        #get our initial user reviews from the review matrix
        user2_ratings = self._R.iloc[user1_index+1:].as_matrix().copy()
        user1_base_ratings = self._R.iloc[user1_index].as_matrix().copy()
        
        #we need a matrix for user1 review ratings. Each row is initialized to user1 reviews, below we zero out all
        #games that user1 and the user2 for that row haven't both reviewed.
        user1_ratings =  np.tile(user1_base_ratings, (user2_ratings.shape[0], 1))
        
        #find where user1 and user2s have reviewed items. (left half is a bool vector, right is a bool matrix).
        #End matrix has each row as a boolean vector showing which items both user1 and the user2 for that row reviewed  
        #We're reversing since we want False where both users reviewed the same item, and True elsewhere.
        mask = (np.isnan(user1_base_ratings) | np.isnan(user2_ratings))

        #Zero out all entries user 1 and 2 for each row pair haven't both reviewed. (Reviews have already been 
        #mean normalized, so this won't throw off the corr calculation)    
        user2_ratings[mask] = 0.0
        user1_ratings[mask] = 0.0
        
        #to pevent overly high similarities between users with few reviews in common, a dampening factor of
        #min(N/50,1) is applied, where N is the number of items both have reviewed.
        num_items = np.sum(~mask,axis=1)
        dampening = (num_items/self.pearson_threshold).clip(0,1)
        
        #pearsonr is cov(1,2)/sqrt(Var(1)*Var(2)). This is just a vectorized implementation, doing row-wise dot 
        #products between two vectors.
        variance1 = (user1_ratings * user1_ratings).sum(axis=1)
        variance2 = (user2_ratings * user2_ratings).sum(axis=1)
        denom = np.sqrt(variance1*variance2)
        covariance = (user1_ratings * user2_ratings).sum(axis=1)

        #catch any divide by 0 errors. Any user2s with 0 variance will produce a NaN.
        with np.errstate(divide='ignore',invalid='ignore'):
            ret = (dampening*covariance)/denom
            ret[np.isnan(ret)] = 0.0
            
        return ret
        
    def _cosine_similarity(self):
        pass
    
    def predict(self,X):
        """Calculates predicted reviews for user,gameID feature pairs X
        ----------
        X : array-like, shape (n_units, 2)
        
        Returns
        -------
        ret : array of float, shape = [n_units]. 
            corresponds to the ordered predictions for each feature pair in X.
        """
        #add a new column to X for the predictions we're about to make.
        output = pd.concat([X,pd.DataFrame(data=np.full(X.shape[0],np.nan),
                                           columns=['rating'],index=X.index)],axis=1)
        
        #flag to account for pd.DataFrame.apply running twice the first iteration and messing up my code
        self._apply_first_flag = True
        
        #group by UserID and predict for all games at once for each user. 
        output = output.groupby('UserID').apply(self.predict_by_user_vector)
        return output.rating.values
        

    def predict_by_user_vector(self, row):
        """Calculates predicted reviews for user,gameID feature pairs X
        ----------
        X : pandas df, shape (n_games, 3), UserID, gameID, rating
        
        Returns
        -------
        row : pandas df, shape (n_games, 3)
            input df with rating column filled in with prediction values
        """
        user = row.iloc[0,0]
        
        #pd.DataFrame.apply runs the first group twice to see if it can optimize. If it's already ran on the current
        #user (e.g, the second iteration of the loop) then skip.
        if self._apply_first_flag:
            self._apply_first_flag = False
            return

        user_average = self._user_average_rating[user]
        
        #Users with 0 standard deviation should automatically predict their average for any game. This make intuitive
        #sense, but it also breaks my code if it's left to run with this edge case.
        if self._user_standard_deviation[user] == 0.0:
            row.rating = user_average
            return row
        
        N = self.N_similar
        
        #this is a matrix where each row is for one of the games we're predicting, and each column is a user.
        games_reviewed = self._R.loc[:,row.gameID].as_matrix().T.copy()
        
        #matrix of masks to get users that have reviewed a given game.
        who_reviewed_which_games = ~np.isnan(games_reviewed)
        
        #get the similarity matrix. Each row is the user's similarity to all other users, with all the users who 
        #didn't review that row's gameID set to zero (so we don't count them)
        user_similarity_vector = self._S.loc[user,:].as_matrix().copy()
        user_similarity_matrix = np.tile(user_similarity_vector,(row.gameID.size,1))
        user_similarity_matrix[~who_reviewed_which_games] = 0.0

        number_of_games = row.gameID.size
        
        #get the indices for the N users with the highest similarity that also reviewed each game.
        
        if self.use_negative_similarities:
            index = np.argpartition(abs(user_similarity_matrix),-N,axis=1)[:,-N:]
        else:
            index = np.argpartition(user_similarity_matrix,-N,axis=1)[:,-N:]
        
        #Use the indices to get the games x N matrix where each row is the user similarities for a given game.
        similarity_matrix = user_similarity_matrix[np.arange(number_of_games).reshape(number_of_games,1),index]
        #Use the indices to get the games x N matrix where each row is the top N similar users normalized rating for
        #the game that row corresponds to. 
        norm_ratings_matrix = games_reviewed[np.arange(number_of_games).reshape(number_of_games,1),index]
        
        user_std = self._get_user_std(user)
                
        #For some N_nearest values, unusual users, etc. it may be that there aren't enough similar users
        #who have reviewed a given game. This basically sets the N_Nearest down to the highest value for
        #that game, user pair before running the final calculation. 
        N = N - np.isnan(norm_ratings_matrix).sum(axis=1)
        norm_ratings_matrix[np.isnan(norm_ratings_matrix)] = 0.0
        
        estimated_user_reviews = user_average + (similarity_matrix * norm_ratings_matrix).sum(axis=1)*(user_std/N)

        row.rating = estimated_user_reviews
        
        return row
  
    def _get_user_std(self,user):
        """Calculates std for user.
        ----------
        user : int, corresponding to a user ID in _R.
        
        Returns
        -------
        user_std : float, precalculated std. If z score normalization is off, then just return 1.0. 
        """
        user_std = 1.0
        if(self.normalize_to_z_scores):
            user_std = self._user_standard_deviation[user]
        return user_std
    

    ####################################################################
    #Naive testing functions. These are simple, non vectorized implementations I used to get a handle on the
    #algorithms, and to test my more complicated optimized implementations. 
    ####################################################################
    def naive_person_r_calc(self,u1,u2):
        u1 = self._R.loc[user1].copy()
        u2 = self._R.loc[user2].copy()

        u1mean = u1.mean()
        u2mean = u2.mean()

        i1 = set(u1.dropna().index.tolist())
        i2 = set(u2.dropna().index.tolist())
        shared = i1.intersection(i2)

        u1 = u1.loc[shared].sort_index()
        u2 = u2.loc[shared].sort_index()

        u1 -= u1mean
        u2 -= u2mean

        return np.dot(u1,u2)/np.sqrt(np.dot(u1,u1)*(np.dot(u2,u2)))
    
    
    def _naive_single_predict_function(self, user, game):
        user_mean = self._user_average_rating[user]
        N = self.N_similar
    
        #nearest users IDs are the index of this series, s(u,u') is the values. It's filtered so only users
        #who have also rated the game in question are considered. 
        N_nearest_similar = self._S[user][self._R[game].notnull()].nlargest(N)
    
        #vectors for most similar user information
        similar_users_means = self._user_average_rating[N_nearest_similar.index].values
        similar_users_item_ratings = self._R.loc[N_nearest_similar.index, game].values
    
        user_std = self._get_user_std(user)
    
        prediction = user_mean + ((user_std/N) * np.dot(N_nearest_similar.values,similar_users_item_ratings))
    
        return prediction



In [6]:
train_X, test_X, train_y,test_y = train_test_split(elite[['UserID','gameID']],elite.rating,test_size=.3,random_state=42)

predictor = U_U_predictor()
predictor.fit(train_X,train_y)

predictor_with_z = U_U_predictor(normalize_to_z_scores=True)
predictor_with_z.fit(train_X,train_y)

def test_user_user_similarity_matrix_vector_function():
    predictor = U_U_predictor()
    predictor._calculate_ratings_matrix(train_X,train_y)
    predictor.fit(train_X,train_y)
    S = pd.read_pickle('user_S')
    
    #making sure that whatever method I'm using to calculate S, that it roughly equals my established baseline using
    #a naive calculation that I picked. 
    assert S.size==np.count_nonzero(np.isclose(S, predictor._S, rtol=1e-07, atol=1e-010, equal_nan=True)),"Similarity matrix was incorrectly calculated with PearsonR."


def test_predict_function(predict):
    naive_test_predict = []
    num_to_test = 10
    for index,row in test_X.iloc[:num_to_test].iterrows():
        naive_prediction = predict._naive_single_predict_function(row.UserID,row.gameID)
        naive_test_predict.append(naive_prediction)
        
    vector_test_predict = predict.predict(test_X.iloc[:num_to_test])

    assert (np.isclose(naive_test_predict,vector_test_predict,rtol=1e-07).all()), "Assert failed with z = {}".format(predictor_with_z.normalize_to_z_scores)
    

def test_for_full_output():
    predictor.fit(train_X,train_y)

    predict = predictor.predict(test_X)

    a = test_X.shape[0]
    b = predict.size

    assert a == b, "there aren't as many predictions as there were rows in the feature set"

test_user_user_similarity_matrix_vector_function()
test_predict_function(predictor)
test_predict_function(predictor_with_z)
test_for_full_output()

### Scratch cell for timing and profiling

In [9]:
#was 189 seconds
#was 97 seconds. 
#was 49 seconds
#calling it good at .8 seconds
#predict is about .6s.

train_X, test_x, train_y,test_y = train_test_split(elite[['UserID','gameID']],elite.rating,test_size=.3,random_state=42)

predictor = U_U_predictor()
#%lprun -f predictor.fit predictor.fit(train_X,train_y)
predictor.fit(train_X,train_y)

%time predictions = predictor.predict(test_x)

CPU times: user 588 ms, sys: 0 ns, total: 588 ms
Wall time: 589 ms


## Model Selection and Error checking

Now that I've gotten some models built out, I can use Sklearn's framework to check out different prediction systems, compare RMSE, and see what kind of model works the best with this dataset.

In [10]:
from sklearn.metrics import mean_squared_error

predictor = Base_Predictor()
train_X, test_x, train_y,test_y = train_test_split(elite[['UserID','gameID']],elite.rating,test_size=.3,random_state=42)
predictor.fit(train_X,train_y)

predictions = predictor.predict(test_x)

mse = mean_squared_error(predictions,test_y)
print(np.sqrt(mse))



1.30322536673


In [7]:
from sklearn.model_selection import GridSearchCV

parameter_candidates = [
  {'normalize_to_z_scores': [True,False]},
  {'N_similar': np.arange(2,21,2), 'use_negative_similarities': [False]},
  {'N_similar': np.arange(8,39,3), 'use_negative_similarities': [True]}
]

clf = GridSearchCV(estimator = U_U_predictor(), param_grid = parameter_candidates, n_jobs=1, 
                   scoring='neg_mean_squared_error', cv=4)

clf.fit(train_X, train_y)

GridSearchCV(cv=4, error_score='raise',
       estimator=U_U_predictor(N_similar=10, normalize_to_z_scores=False, pearson_threshold=50,
       similarity_type='pearson', use_negative_similarities=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'normalize_to_z_scores': [True, False]}, {'N_similar': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20]), 'use_negative_similarities': [False]}, {'N_similar': array([ 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]), 'use_negative_similarities': [True]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [9]:
print('Best score for data1:', -clf.best_score_) 
print('Use Z normalizing?: ',clf.best_estimator_.normalize_to_z_scores) 
print('Best N_similar:',clf.best_estimator_.N_similar)
print('Use use_negative_similarities?: ',clf.best_estimator_.use_negative_similarities)

Best score for data1: 1.73485927576
Use Z normalizing?:  True
Best N_similar: 10
Use use_negative_similarities?:  False


In [118]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(U_U_predictor(normalize_to_z_scores=True,N_similar=10), 
                         train_X, train_y, cv=5, scoring='neg_mean_squared_error')

In [20]:
from sklearn.metrics import mean_squared_error

predictor = U_U_predictor(normalize_to_z_scores=True,N_similar=10, use_negative_similarities=False)
train_X, test_x, train_y,test_y = train_test_split(elite[['UserID','gameID']],elite.rating,test_size=.3,random_state=42)
predictor.fit(train_X,train_y)

predictions = predictor.predict(test_x)

mse = mean_squared_error(predictions,test_y)
print(np.sqrt(mse))



1.25584900521


In [121]:
scores = cross_val_score(Base_Predictor(), train_X, train_y, cv=5, scoring='neg_mean_squared_error')

In [123]:
-(scores.mean())

1.7040152706232921